In [6]:
import pandas as pd 
import numpy as np
import glob
import os
import math
import scipy.stats as stats
from scipy.stats import pearsonr
from sklearn.decomposition import FactorAnalysis
import pingouin as pg
import semopy
import statsmodels.api as sm
from sklearn import linear_model
from mlxtend.feature_selection import SequentialFeatureSelector

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
# Read CSV
tam_df = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/tam_results/tam.csv')
demographics = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/demographic_results/demographics_processed.csv')

In [8]:
# Merge
tam_df = pd.merge(tam_df, demographics[['user_id', 'user_archetype', 'age', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']])
# Move columns 
col = tam_df.pop('q28')
tam_df.insert(23, col.name, col)
col = tam_df.pop('q29')
tam_df.insert(24, col.name, col)
'''# To int
tam_df['q28'] = tam_df['q28'].fillna(np.nan)
tam_df['q29'] = tam_df['q29'].fillna(np.nan)
tam_df['q28'] = tam_df['q28'].astype(int)
tam_df['q29'] = tam_df['q29'].astype(int)'''
# Update column names
#AA_1
tam_df.columns = ['user_id',
 'user_archetype',
 'ai_attitude',
 'PU1',
 'PU2',
 'PU3',
 'PU4',
 'PU5',
 'PU6',
 'PU7',
 'PU8',
 'PEOU1',
 'PU9',
 'PEOU2',
 'SE1',
 'SE2',
 'voluntary',
 'SN1',
 'SN2',
 'prestige_image',
 'PU10',
 'PU11',
 'BC1',
 'BC2',
 'BC3',
 'clinical_workflow',
 'infrastructure',
 'intention',
 'colleagues',
 'PU12',
 'final_comments',
 'age_new',
 'sex_new',
 'medical_speciality_new',
 'grade_new',
 'ai_familiarity_new']

 # Get result columns 
tam_df_results = tam_df.iloc[:,2:30]

"# To int\ntam_df['q28'] = tam_df['q28'].fillna(np.nan)\ntam_df['q29'] = tam_df['q29'].fillna(np.nan)\ntam_df['q28'] = tam_df['q28'].astype(int)\ntam_df['q29'] = tam_df['q29'].astype(int)"

In [27]:
def cronbach_alpha_fun(data):
    # Ensure the data is a pandas DataFrame
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data should be a pandas DataFrame")

    # Calculate the number of items (variables)
    N = data.shape[1]
    
    # Calculate the correlation matrix
    df_corr = data.corr()
    
    # Extract the lower triangle of the correlation matrix excluding the diagonal
    lower_triangular_corr = df_corr.where(np.tril(np.ones(df_corr.shape), k=-1).astype(bool))
    
    # Calculate the mean of the lower triangular correlation coefficients
    mean_r = lower_triangular_corr.stack().mean()
    
    # Calculate Cronbach's Alpha using the formula
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    
    return cronbach_alpha


def cronbach_alpha_fun2(data):
    # Number of items
    N = data.shape[1]
    
    # Compute the variance for each item
    item_variances = data.var(axis=0, ddof=1)
    
    # Compute the total variance of the sum of all items
    total_variance = data.sum(axis=1).var(ddof=1)
    
    # Calculate Cronbach's Alpha
    alpha = (N / (N - 1)) * (1 - (item_variances.sum() / total_variance))
    
    return alpha

In [11]:
tam_df_results.columns

Index(['ai_attitude', 'PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8',
       'PEOU1', 'PU9', 'PEOU2', 'SE1', 'SE2', 'voluntary', 'SN1', 'SN2',
       'prestige_image', 'PU10', 'PU11', 'BC1', 'BC2', 'BC3',
       'clinical_workflow', 'infrastructure', 'intention', 'colleagues',
       'PU12'],
      dtype='object')

In [12]:
# Get groups'
#tam_groups = ['AA', 'REL', 'PU', 'PEOU', 'SE', 'VOL', 'SN', 'IMG', 'QUAL', 'BC', 'FAC', 'BI', 'DN', 'US', 'TBC'] # old
tam_groups = ['PU', 'PEOU', 'SE', 'SN', 'BC']

In [92]:
pg.cronbach_alpha(tam_df_results[['AA2_TBC4', 'AA3_PU5', 'REL1', 'REL2', 'PU1', 'PU2', 'PU3', 'PU4', 'TBC2', 'QUAL1', 'QUAL2', 'US1']])

(0.8728371578057946, array([0.795, 0.93 ]))

In [110]:
pg.cronbach_alpha(tam_df_results[['PEOU1', 'PEOU3']])

(0.6029246344206973, array([0.176, 0.809]))

In [205]:
pg.cronbach_alpha(tam_df_results[['SE1', 'SE2']])

(0.4397163120567378, array([-0.162,  0.73 ]))

In [204]:
pg.cronbach_alpha(tam_df_results[['SN2', 'SN1']])

(0.06511627906976725, array([-0.939,  0.549]))

In [31]:
pg.cronbach_alpha(tam_df_results[['BC1', 'BC2', 'BC3']])

(0.49899396378269634, array([0.099, 0.739]))

In [33]:
pg.cronbach_alpha(tam_df_results[['BC2', 'BC3']])

(0.30015313935681487, array([-0.434,  0.658]))

In [13]:
# Mean and std
tam_df_mean_std = pd.DataFrame()
tam_df_mean_std['mean'] = tam_df_results.mean()
tam_df_mean_std['std'] = tam_df_results.std()
tam_df_mean_std

,mean,std
ai_attitude,3.59375,0.712079
PU1,3.71875,0.581121
PU2,2.93750,0.840027
PU3,3.78125,0.870090
PU4,3.87500,1.039541
PU5,3.75000,0.879883
PU6,3.37500,1.039541
PU7,3.53125,1.106706
PU8,3.62500,0.751343
PEOU1,3.65625,0.937029


In [14]:
# Cronbach's alpha
for group in tam_groups:
    group_df = tam_df_results.filter(regex=group)
    if group_df.shape[1] > 1:
        cronbach_alpha = pg.cronbach_alpha(group_df)
        print(group_df.columns)
        print(f"Cronbach's alpha for {group}:", cronbach_alpha)

Index(['PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10',
       'PU11', 'PU12'],
      dtype='object')
Cronbach's alpha for PU: (0.8702795962462511, array([0.792, 0.928]))
Index(['PEOU1', 'PEOU2'], dtype='object')
Cronbach's alpha for PEOU: (0.607915567282322, array([0.197, 0.809]))
Index(['SE1', 'SE2'], dtype='object')
Cronbach's alpha for SE: (0.41025641025641035, array([-0.208,  0.712]))
Index(['SN1', 'SN2'], dtype='object')
Cronbach's alpha for SN: (0.08342022940563076, array([-0.878,  0.553]))
Index(['BC1', 'BC2', 'BC3'], dtype='object')
Cronbach's alpha for BC: (0.49899396378269634, array([0.099, 0.739]))


In [101]:
# old
# Cronbach's alpha
for group in tam_groups:
    group_df = tam_df_results.filter(regex=group)
    if group_df.shape[1] > 1:
        cronbach_alpha = pg.cronbach_alpha(group_df)
        print(group_df.columns)
        print(f"Cronbach's alpha for {group}:", cronbach_alpha)

Index(['AA1', 'AA2_TBC4', 'AA3_PU5'], dtype='object')
Cronbach's alpha for AA: (0.29315960912052114, array([-0.283,  0.636]))
Index(['REL1', 'REL2'], dtype='object')
Cronbach's alpha for REL: (0.42163153070577475, array([-0.2  ,  0.721]))
Index(['AA3_PU5', 'PU1', 'PU2', 'PU3', 'PU4'], dtype='object')
Cronbach's alpha for PU: (0.8229149531172891, array([0.701, 0.905]))
Index(['PEOU1', 'PEOU3'], dtype='object')
Cronbach's alpha for PEOU: (0.6029246344206973, array([0.176, 0.809]))
Index(['SE1', 'SE2'], dtype='object')
Cronbach's alpha for SE: (0.4397163120567378, array([-0.162,  0.73 ]))
Index(['SN1', 'SN2'], dtype='object')
Cronbach's alpha for SN: (0.06511627906976725, array([-0.939,  0.549]))
Index(['QUAL1', 'QUAL2'], dtype='object')
Cronbach's alpha for QUAL: (0.5751633986928104, array([0.119, 0.795]))
Index(['AA2_TBC4', 'TBC2', 'BC1', 'BC2', 'BC3', 'FAC1_BC4'], dtype='object')
Cronbach's alpha for BC: (0.6444979304602348, array([0.408, 0.808]))
Index(['FAC1_BC4', 'FAC2'], dtype='obj

In [54]:
# Inter-Item Correlations
for group in tam_groups:
    group_df = tam_df.filter(regex=group)
    if group_df.shape[1] > 1:
        inter_item_corr = group_df.corr()
        print(f"Inter-Item Correlations for {group}:\n", inter_item_corr)

Inter-Item Correlations for AA:
                AA1  AA2_TBC4   AA3_PU5
AA1       1.000000  0.253833  0.062974
AA2_TBC4  0.253833  1.000000  0.094169
AA3_PU5   0.062974  0.094169  1.000000
Inter-Item Correlations for REL:
           REL1      REL2
REL1  1.000000  0.271451
REL2  0.271451  1.000000
Inter-Item Correlations for PU:
           AA3_PU5       PU1       PU2       PU3       PU4
AA3_PU5  1.000000  0.152267  0.286556  0.453601  0.524872
PU1      0.152267  1.000000  0.598267  0.603172  0.485898
PU2      0.286556  0.598267  1.000000  0.773066  0.344568
PU3      0.453601  0.603172  0.773066  1.000000  0.554586
PU4      0.524872  0.485898  0.344568  0.554586  1.000000
Inter-Item Correlations for PEOU:
                   PEOU1     PEOU3     SE1_PEOU5
PEOU1      1.000000e+00  0.461192 -8.531226e-17
PEOU3      4.611924e-01  1.000000  1.388474e-01
SE1_PEOU5 -8.531226e-17  0.138847  1.000000e+00
Inter-Item Correlations for SE:
            SE1_PEOU5       SE2
SE1_PEOU5   1.000000  0.313211

In [65]:
# Item-Total Correlations
for group in tam_groups:
    group_df = tam_df.filter(regex=group)
    if group_df.shape[1] > 1:
        #total = group_df.sum(axis=1)
        #item_total_corr = group_df.apply(lambda x: x.corr(total))
        item_total_corr = group_df.apply(lambda x: x.corr(group_df.drop(columns=x.name).sum(axis=1)))
        print(f"Item-Total Correlations for {group}:\n", item_total_corr)

Item-Total Correlations for AA:
 AA1         0.187802
AA2_TBC4    0.228628
AA3_PU5     0.096964
dtype: float64
Item-Total Correlations for REL:
 REL1    0.271451
REL2    0.271451
dtype: float64
Item-Total Correlations for PU:
 AA3_PU5    0.424317
PU1        0.599784
PU2        0.676299
PU3        0.818114
PU4        0.600544
dtype: float64
Item-Total Correlations for PEOU:
 PEOU1        0.381335
PEOU3        0.480308
SE1_PEOU5    0.066021
dtype: float64
Item-Total Correlations for SE:
 SE1_PEOU5    0.313211
SE2          0.313211
dtype: float64
Item-Total Correlations for SN:
 SN1    0.035313
SN2    0.035313
dtype: float64
Item-Total Correlations for QUAL:
 QUAL1    0.422527
QUAL2    0.422527
dtype: float64
Item-Total Correlations for BC:
 AA2_TBC4    0.164718
TBC2        0.159555
BC1         0.261657
BC2         0.506411
BC3         0.206464
FAC1_BC4    0.116350
dtype: float64
Item-Total Correlations for FAC:
 FAC1_BC4    0.007879
FAC2        0.007879
dtype: float64
Item-Total Correlat

In [209]:
fa = FactorAnalysis(n_components=3)  # Number of factors can be adjusted based on theory
fa.fit(tam_df_results[['AA2_TBC4', 'AA3_PU5', 'REL1', 'REL2', 'PU1', 'PU2', 'PU3', 'PU4', 'TBC2', 'QUAL1', 'QUAL2', 'US1']])

# Factor loadings
factor_loadings = pd.DataFrame(fa.components_.T, columns=[f'Factor{i+1}' for i in range(fa.n_components)], index=tam_df_results[['AA2_TBC4', 'AA3_PU5', 'REL1', 'REL2', 'PU1', 'PU2', 'PU3', 'PU4', 'TBC2', 'QUAL1', 'QUAL2', 'US1']].columns)
#print("Factor Loadings:\n", factor_loadings)

# Define threshold for significant loadings
threshold = 0.4

# Highlight significant loadings
significant_loadings = factor_loadings.map(lambda x: '*' if abs(x) >= threshold else '')
significant_loadings

FactorAnalysis(n_components=3)

,Factor1,Factor2,Factor3
AA2_TBC4,,,
AA3_PU5,*,,
REL1,*,*,
REL2,,,
PU1,*,,
PU2,*,*,
PU3,*,*,
PU4,*,,
TBC2,*,,
QUAL1,,,


In [128]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
tam_df_results_imputed = pd.DataFrame(imputer.fit_transform(tam_df_results), columns=tam_df_results.columns)
tam_df_results_imputed = (tam_df_results_imputed + 0.01).round() # Add tiny ammount to avoid weird rounding of .5
tam_df_results_imputed = tam_df_results_imputed.astype(int)

In [88]:


fa = FactorAnalysis(n_components=9)  # Number of factors can be adjusted based on theory
fa.fit(tam_df_results_imputed)

# Factor loadings
factor_loadings = pd.DataFrame(fa.components_.T, columns=[f'Factor{i+1}' for i in range(fa.n_components)], index=tam_df_results_imputed.columns)
print("Factor Loadings:\n", factor_loadings)

FactorAnalysis(n_components=9)

Factor Loadings:
            Factor1   Factor2   Factor3   Factor4   Factor5   Factor6  \
AA1       0.207330 -0.115548 -0.177429 -0.304826 -0.077782 -0.088306   
AA2_TBC4  0.223415 -0.222190 -0.032044 -0.056206 -0.257339 -0.022256   
AA3_PU5   0.694230  0.417396  0.180505  0.093632  0.024758  0.000778   
REL1      0.573749 -0.137303  0.117722 -0.150416 -0.139797  0.244769   
REL2      0.161550  0.017691 -0.391395 -0.203240 -0.028692  0.209814   
PU1       0.435362 -0.328990 -0.208703  0.024109 -0.524052  0.127235   
PU2       0.579973 -0.234555 -0.094493 -0.259770 -0.188863 -0.031272   
PU3       0.788658 -0.133299 -0.255851 -0.112584 -0.162815 -0.203781   
PU4       0.481915  0.029751 -0.133007  0.132747 -0.205996 -0.069745   
PEOU1     0.526574 -0.256817 -0.585682  0.100030  0.327814  0.225633   
TBC2      0.437788 -0.151852 -0.394982 -0.054893  0.056079 -0.604107   
PEOU3     0.172205 -0.135869 -0.141606 -0.114248  0.080007  0.221819   
SE1       0.067352 -0.073932  0.085038 -0.0203

In [201]:
# Define threshold for significant loadings
threshold = 0.4

# Highlight significant loadings
significant_loadings = factor_loadings.map(lambda x: '*' if abs(x) >= threshold else '')
significant_loadings

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9
AA1,,,,,,,,,
AA2_TBC4,,,,,,,,,
AA3_PU5,*,*,,,,,,,
REL1,*,,,,,,,,
REL2,,,,,,,,,
PU1,*,,,,*,,,,
PU2,*,,,,,,*,,
PU3,*,,,,,,*,,
PU4,*,,,,,,,,
PEOU1,*,,*,,,,,,


In [36]:
# Define threshold for significant loadings
threshold = -0.4

# Highlight significant loadings
significant_loadings = factor_loadings.map(lambda x: '*' if abs(x) <= threshold else '')
significant_loadings

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9
AA1,,,,,,,,,
AA2,,,,,,,,,
AA3_PU5,,,,,,,,,
REL1,,,,,,,,,
REL2,,,,,,,,,
PU1,,,,,,,,,
PU2,,,,,,,,,
PU3,,,,,,,,,
PU4,,,,,,,,,
PEOU1,,,,,,,,,


In [155]:
def next_possible_feature (X_npf, y_npf, current_features, ignore_features=[]):
    '''
    This function will loop through each column that isn't in your feature model and 
    calculate the r-squared value if it were the next feature added to your model. 
    It will display a dataframe with a sorted r-squared value.
    X_npf = X dataframe
    y_npf = y dataframe
    current_features = list of features that are already in your model
    ignore_features = list of unused features we want to skip over
    '''   
    #Create an empty dictionary that will be used to store our results
    function_dict = {'predictor': [], 'r-squared':[]}
    #Iterate through every column in X
    for col in X_npf.columns:
        #But only create a model if the feature isn't already selected or ignored
        if col not in (current_features+ignore_features):
            #Create a dataframe called function_X with our current features + 1
            selected_X = X_npf[current_features + [col]]
            #Fit a model for our target and our selected columns 
            model = sm.OLS(y_npf, sm.add_constant(selected_X)).fit()
            #Predict what  our target would be for our selected columns
            y_preds = model.predict(sm.add_constant(selected_X))
            #Add the column name to our dictionary
            function_dict['predictor'].append(col)
            #Calculate the r-squared value between the target and predicted target
            r2 = np.corrcoef(y_npf, y_preds)[0, 1]**2
            #Add the r-squared value to our dictionary
            function_dict['r-squared'].append(r2)
    #Once it's iterated through every column, turn our dict into a sorted DataFrame
    function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'], ascending = False).reset_index(drop=True)
    return function_df['predictor'][0], function_df['r-squared'][0]

In [174]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def check_multicollinearity(X_npf, features):
    #Define an empty dataframe to capture the VIF scores
    vif_2 = pd.DataFrame()
    #Create a Dataframe with only our selected features 
    X_2 = X_npf[features]
    #Label the scores with their related columns
    vif_2["features"] = X_2.columns
    #For each column,run a variance_inflaction_factor against all other columns
    # to get a VIF Factor score
    vif_2["VIF"] = [variance_inflation_factor(X_2.values, i) \
                    for i in range(len(X_2.columns))]
    if vif_2['VIF'].iloc[-1] > 5:
        return True
    else:
        return False

In [196]:
#Split our DataFrame into X (input) and y (output)
y = tam_df_results_imputed['BI1']
X = tam_df_results_imputed.drop('BI1', axis = 1)

# Feature lists
selected_features = []
features_to_ignore = []
r2_list = []
n_iterations = len(tam_df_results_imputed.columns) - 2

#Create an empty dictionary that will be used to store our results
function_dict = {'predictor': [], 'r-squared':[]}
#Iterate through every column in X
for col in X.columns:
    #Create a dataframe called selected_X with only the 1 column
    selected_X = X[[col]]
    #Fit a model for our target and our selected column 
    model = sm.OLS(y, sm.add_constant(selected_X)).fit()
    #Predict what our target would be for our model
    y_preds = model.predict(sm.add_constant(selected_X))
    #Add the column name to our dictionary
    function_dict['predictor'].append(col)
    #Calculate the r-squared value between the target and predicted target
    r2 = np.corrcoef(y, y_preds)[0, 1]**2
    #Add the r-squared value to our dictionary
    function_dict['r-squared'].append(r2)
    
#Once it's iterated through every column, turn our dictionary into a DataFrame and sort it
function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'], ascending = False).reset_index(drop=True)
selected_features.append(function_df['predictor'][0])
r2_list.append(function_df['r-squared'][0])
for n in range(n_iterations):
    next_predictor, next_rsquared = next_possible_feature(X_npf=X, y_npf=y, current_features=selected_features, ignore_features=features_to_ignore)    
    temp_feature_list = selected_features.copy()
    temp_feature_list.append(next_predictor)
    multicollinearity_bool = check_multicollinearity(X, temp_feature_list)
    if multicollinearity_bool == True:
        features_to_ignore.append(next_predictor)
    elif multicollinearity_bool == False:
        selected_features.append(next_predictor)
        r2_list.append(function_df['r-squared'][0])


In [197]:
temp_feature_list

['PU1', 'FAC2']

In [198]:
features_to_ignore

['US1',
 'PU4',
 'AA3_PU5',
 'REL1',
 'BC3',
 'AA1',
 'VOL1',
 'AA2_TBC4',
 'BC2',
 'QUAL2',
 'SN1',
 'DN1',
 'PU3',
 'FAC1_BC4',
 'PEOU1',
 'PU2',
 'SE2',
 'QUAL1',
 'SE1',
 'IMG1',
 'TBC2',
 'SN2',
 'PEOU3',
 'REL2',
 'BC1',
 'FAC2']

In [199]:
selected_features

['PU1']

In [200]:
r2_list

[0.7493261455525617]

# test

In [46]:
data = {
    'AA1': [4, 3, 5, 4, 3],
    'AA2': [3, 4, 4, 3, 4],
    'REL1': [5, 4, 4, 5, 3],
    'REL2': [4, 5, 4, 4, 4],
    'PU1': [5, 4, 5, 4, 3],
    # Add all other questions similarly
}

test_df = pd.DataFrame(data)


In [47]:
fa = FactorAnalysis(n_components=2)
fa.fit(test_df)
print("Factor Loadings:\n", fa.components_)

FactorAnalysis(n_components=2)

Factor Loadings:
 [[-0.43247705  0.41145629 -0.711514    0.12190896 -0.57211539]
 [ 0.40775551  0.24935113 -0.1065351   0.06361384  0.46087833]]


In [51]:
from semopy import Model, Optimizer

# Define the model structure
model_desc = """
AA =~ AA1 + AA2
REL =~ REL1 + REL2
PU =~ PU1
"""

model = Model(model_desc)
sem_res = model.fit(test_df)
print("CFA Results:\n", sem_res)

CFA Results:
 Name of objective: MLW
Optimization method: SLSQP
Inequality constraints incompatible
Objective value: 23.257
Number of iterations: 83
Params: -1.394 -0.833 0.493 0.555 0.811 0.419 0.195 0.585 0.785 0.281 1.030 0.212 0.177


In [50]:
# Define the SEM structure
sem_desc = """
BI =~ AA + PU + REL
AA =~ AA1 + AA2
REL =~ REL1 + REL2
PU =~ PU1
"""

sem_model = Model(sem_desc)
sem_res = sem_model.fit(test_df)
print("SEM Results:\n", sem_res)

SEM Results:
 Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 38.813
Number of iterations: 26
Params: 1.114 0.833 -0.350 -0.230 0.000 0.179 0.193 0.381 0.030 0.057 0.000 0.295 0.146
